In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [6]:
## new fix for data import
import os
os.environ["HTTP_PROXY"] = ""
os.environ["HTTPS_PROXY"] = ""
os.environ["NO_PROXY"] = ""

In [7]:
## Input Data
chase = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/Chase_11449/" + "chase_stacked_final.parquet")
print(chase.shape)

tmobile = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/tmobile_10823/" + "tmbile_file_to_use.parquet")
print(tmobile.shape)

stripe = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/stripe_11363/" + "stripe_file_to_use.parquet")
print(stripe.shape)

fleector = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/fleector_11124/" + "fleector_sample_to_use.parquet")
print(fleector.shape)

(640958, 36)
(114460, 39)
(75000, 22)
(83704, 92)


In [8]:
## keep seleid, full address, phone, taxid, performance flag >> business information >> might need to dedup

#### SBFE Inquiry

In [9]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Data Modeling Clean/sbfe_seleid_append_deduped_cleaned_0501.parquet"
sbfe = pd.read_adls(path, reader = pd.read_parquet)
print(sbfe.shape)

(79429086, 20)


In [10]:
sbfe = sbfe[["seleid", "historydate_x"]]
sbfe.columns = ["seleid_inquiry_sbfe", "inquiry_date"]

In [11]:
sbfe["inquiry_date"] = sbfe["inquiry_date"].astype(int)
sbfe["inquiry_date_fmt"] = pd.to_datetime(sbfe["inquiry_date"], format='%Y%m%d')

In [12]:
sbfe["seleid_inquiry_sbfe"] = sbfe["seleid_inquiry_sbfe"].astype(float)

In [13]:
sbfe.head()

,seleid_inquiry_sbfe,inquiry_date,inquiry_date_fmt
0,1.336156e+11,20210701,2021-07-01
1,1.330122e+09,20210701,2021-07-01
2,9.064136e+08,20210701,2021-07-01
5,6.136772e+07,20210701,2021-07-01
6,1.371452e+11,20210701,2021-07-01


#### Stripe 11363

In [28]:
stripe.columns

Index(['Unnamed: 0', 'uniqueid', 'historydate', 'companyname',
       'alternatecompanyname', 'bus_streetaddress1', 'bus_streetaddress2',
       'bus_city', 'bus_state', 'bus_zip', 'bus_phone', 'bus_website',
       'firstname', 'lastname', 'streetaddress1', 'streetaddress2', 'city',
       'state', 'zip', 'email', 'ip_address', 'performance_indicator'],
      dtype='object')

In [29]:
stripe = stripe.fillna("")
stripe = stripe.drop_duplicates(subset = ['companyname', 'alternatecompanyname', 'bus_streetaddress1', 
                                          'bus_streetaddress2', 'bus_city', 'bus_state', 'bus_zip', 'bus_phone', 'bus_website'], ignore_index = True) 
print(stripe.shape)
print(stripe.uniqueid.is_unique)

(74443, 22)
True


In [30]:
# Compile SELEIDS
stripe_sba = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/stripe_11363/final_str_11363_20221025file_sba21_nonsbfe.csv.gz",
                      compression = "gzip", reader = pd.read_csv)
print(stripe_sba.shape)
stripe_sba = stripe_sba.drop_duplicates(subset = ["uniqueid"], ignore_index = True) 
print(stripe_sba.shape)

(249000, 391)
(248318, 391)


In [31]:
keep = ['uniqueid', 'lnlexidsele',  'inquirycount03m', 'inquirycount12m', 'inquirycreditcount03m', 'inquirycreditcount12m',]
stripe_sba = stripe_sba[keep]
stripe = stripe.merge(stripe_sba, on = "uniqueid")
print(stripe.shape)

(74443, 27)


In [33]:
freq(stripe.historydate.str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
historydate,,,,
2022-01,24426.0,0.328117,24426.0,0.328117
2022-02,24209.0,0.325202,48635.0,0.653319
2022-03,25808.0,0.346681,74443.0,1.000000


In [34]:
freq(stripe.performance_indicator)

,Count,Pct,Cuml Count,Cuml Pct
performance_indicator,,,,
False,73807.0,0.991457,73807.0,0.991457
True,636.0,0.008543,74443.0,1.000000


In [38]:
stripe["bad"] = np.where(stripe.performance_indicator == "True", 1, 0)

In [39]:
fmt = make_format(cuts = [-np.inf, 0, np.inf])
bivariate(stripe.lnlexidsele, stripe.bad, format = fmt, observed = True)

tag              bad                                               
stats              N      PctN    Sum      Mean       WoE        IV
lnlexidsele                                                        
<= 0         35861.0  0.481724  357.0  0.009955  0.153566  0.012261
1+           38555.0  0.517913  279.0  0.007236 -0.168135  0.013494
Missing         27.0  0.000363    0.0  0.000000  0.000000 -0.000000
Total        74443.0  1.000000  636.0  0.008543  0.000000  0.025756

In [40]:
stripe.to_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/with_seleid/stripe_with_seleid.parquet")

In [41]:
stripe = stripe[stripe["lnlexidsele"] > 0]
print(stripe.shape)

(38555, 28)


In [57]:
stripe["credit_flag_cutoff_date"] = stripe["historydate"].str.slice(0, 10)
stripe["credit_flag_cutoff_date_fmt"] = pd.to_datetime(stripe["credit_flag_cutoff_date"], format='%Y-%m-%d')

In [58]:
sbfe_for_stripe = sbfe.merge(stripe, left_on = "seleid_inquiry_sbfe", right_on = "lnlexidsele")
print(sbfe_for_stripe.shape)
sbfe_for_stripe = sbfe_for_stripe[sbfe_for_stripe["inquiry_date_fmt"] < sbfe_for_stripe["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_stripe.shape)

sbfe_for_stripe['inquiry_count_06']   = np.where((sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6, 1, 0)
sbfe_for_stripe['inquiry_count_03']   = np.where((sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3, 1, 0)
sbfe_for_stripe['inquiry_count_01']   = np.where((sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1, 1, 0)
sbfe_for_stripe['inquiry_count_week'] = np.where((sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1, 1, 0)

sbfe_for_stripe_roll = sbfe_for_stripe.groupby(by = ['uniqueid'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()
print(sbfe_for_stripe_roll.shape)

(76511, 33)
(49457, 33)
(7769, 5)


/tmp/ipykernel_9607/3733929883.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sbfe_for_stripe_roll = sbfe_for_stripe.groupby(by = ['uniqueid'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()


In [60]:
stripe = stripe.merge(sbfe_for_stripe_roll, on = "uniqueid", how = "left")
print(stripe.shape)

(38555, 34)


In [61]:
stripe['inquiry_count_06'] = np.where(stripe["inquiry_count_06"].isnull(), 0,   stripe["inquiry_count_06"])
stripe['inquiry_count_03'] = np.where(stripe["inquiry_count_03"].isnull(), 0,   stripe["inquiry_count_03"])
stripe['inquiry_count_01'] = np.where(stripe["inquiry_count_01"].isnull(), 0,   stripe["inquiry_count_01"])
stripe['inquiry_count_week']=np.where(stripe["inquiry_count_week"].isnull(), 0, stripe["inquiry_count_week"])

In [62]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
keep = ['inquirycount03m', 'inquirycount12m', 'inquirycreditcount03m', 'inquirycreditcount12m', "inquiry_count_06", 'inquiry_count_03', 'inquiry_count_01', 'inquiry_count_week']
for i, a in enumerate(keep):
    result[i] = bivariate(stripe[a], stripe["bad"], format = fmt)

wb = TableWriter(filename = "./_temp/stripe.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

In [2]:
stripe = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/with_seleid/stripe_with_seleid_and_inquiry.parquet")
print(stripe.shape)

(38555, 34)


In [3]:
fmt = make_format(cuts = [-np.inf, -1,  0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, np.inf])
bivariate(stripe["inquiry_count_06"], stripe["bad"], format = fmt, observed = True)

tag                   bad                                               
stats                   N      PctN    Sum      Mean       WoE        IV
inquiry_count_06                                                        
0                 31357.0  0.813306  221.0  0.007048 -0.028368  0.000645
1                  2622.0  0.068007   33.0  0.012586  0.557070  0.028092
2                   789.0  0.020464    4.0  0.005070 -0.359800  0.002230
3                   347.0  0.009000    4.0  0.011527  0.468153  0.002505
4                   340.0  0.008819    2.0  0.005882 -0.210310  0.000352
5                   515.0  0.013358    2.0  0.003883 -0.627540  0.003920
6                   455.0  0.011801    3.0  0.006593 -0.095481  0.000103
7                   192.0  0.004980    0.0  0.000000  0.000000 -0.000000
8                    99.0  0.002568    0.0  0.000000  0.000000 -0.000000
9                    63.0  0.001634    0.0  0.000000  0.000000 -0.000000
10                   64.0  0.001660    0.0  0.000000  0.000000 -0.000000
11+                1712.0  0.044404   10.0  0.005841 -0.217385  0.001889
Total             38555.0  1.000000  279.0  0.007236  0.000000  0.039736

In [76]:
stripe.to_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/with_seleid/stripe_with_seleid_and_inquiry.parquet")

#### TMobile 10823

In [9]:
tmobile.shape

(114460, 39)

In [10]:
tmobile["credit_flag_cutoff_date"] = tmobile.appdatetime_gmt.str.slice(0, 9)
tmobile["credit_flag_cutoff_date_fmt"] = pd.to_datetime(tmobile["credit_flag_cutoff_date"], format='%d%b%Y')

In [11]:
freq(tmobile["credit_flag_cutoff_date_fmt"].astype(str).str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
credit_flag_cutoff_date_fmt,,,,
2022-01,20510.0,0.179189,20510.0,0.179189
2022-02,22452.0,0.196156,42962.0,0.375345
2022-03,22846.0,0.199598,65808.0,0.574943
2022-04,14973.0,0.130814,80781.0,0.705757
2022-05,16271.0,0.142154,97052.0,0.847912
2022-06,17408.0,0.152088,114460.0,1.000000


In [12]:
freq(tmobile.src)

,Count,Pct,Cuml Count,Cuml Pct
src,,,,
march,33679.0,0.294243,33679.0,0.294243
retro,80781.0,0.705757,114460.0,1.000000


In [13]:
tmobile.columns

Index(['crid_encr', 'appdatetime_gmt', 'businessname', 'business_contactphone',
       'business_addressline1', 'business_addressline2',
       'business_addresscity', 'business_addressstate',
       'business_addresspostalcode', 'federaltaxid', 'contactfirstname',
       'contactmiddlename', 'contactlastname', 'contact_contactphone',
       'contact_addressline1', 'contact_addressline2', 'contact_addresscity',
       'contact_addressstate', 'contact_addresspostalcode', 'contact_dob',
       'contact_idnum', 'contact_idtype', 'contact_idstate', 'contact_email',
       'application_channel', 'fraud_ind', 'fraud_tp_ind', 'fraud_fp_ind',
       'fpd_ind', 'any_wo_ind', 'activationflag', 'seq', 'account', 'date',
       'src', 'contact_dob_year', 'contact_dob_month', 'contact_dob_day',
       'contact_driverlicensenumber', 'credit_flag_cutoff_date',
       'credit_flag_cutoff_date_fmt'],
      dtype='object')

In [14]:
freq(tmobile.fraud_ind)

,Count,Pct,Cuml Count,Cuml Pct
fraud_ind,,,,
0,58655.0,0.512450,58655.0,0.512450
1,3762.0,0.032867,62417.0,0.545317
Missing,52043.0,0.454683,114460.0,1.000000


In [15]:
tmobile = tmobile[tmobile["fraud_ind"].notnull()]
print(tmobile.shape)

(62417, 41)


In [16]:
freq(tmobile.src)

,Count,Pct,Cuml Count,Cuml Pct
src,,,,
march,21914.0,0.35109,21914.0,0.35109
retro,40503.0,0.64891,62417.0,1.00000


In [17]:
shell_202209 = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/tmobile_10823/tmo_10823_file2_businessadress_business_shell_v22_fraud_w20220907-145024_sas_layout_busshell.csv", 
                             encoding = 'iso-8859-1')
print(shell_202209.shape)
shell_202303 = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/tmobile_10823/tmo_10823_file3_business_shell_v22_fraud__businessaddressw20230310-100953_sas_layout_busshell.csv", 
                             encoding = 'iso-8859-1')
print(shell_202303.shape)

/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:235: DtypeWarning: Columns (22,54,605,654,655,656,657,658,659) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


(80781, 2702)


/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:235: DtypeWarning: Columns (20,52,71,592,593,596,603,652,653,654,655,656,657,697,698,699) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


(62203, 2700)


In [18]:
keep = ['account', 'seq', 'id_seleid', 'history_datetime', 'inq_total_count03', 'inq_total_count06', 'inq_total_count12',  'inq_total_count24', 
         'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24',]
temp_march = tmobile[tmobile["src"] == "march"]
print(temp_march.shape)
temp_march = temp_march.merge(shell_202303[keep], on = "account")
print(temp_march.shape)

temp_retro = tmobile[tmobile["src"] != "march"]
print(temp_retro.shape)
temp_retro = temp_retro.merge(shell_202209[keep], on = "account")
print(temp_retro.shape)

(21914, 41)
(21914, 53)
(40503, 41)
(40503, 53)


In [19]:
tmobile = pd.concat([temp_march, temp_retro], ignore_index = True)
print(tmobile.shape)

(62417, 53)


In [22]:
tmobile.columns

Index(['crid_encr', 'appdatetime_gmt', 'businessname', 'business_contactphone',
       'business_addressline1', 'business_addressline2',
       'business_addresscity', 'business_addressstate',
       'business_addresspostalcode', 'federaltaxid', 'contactfirstname',
       'contactmiddlename', 'contactlastname', 'contact_contactphone',
       'contact_addressline1', 'contact_addressline2', 'contact_addresscity',
       'contact_addressstate', 'contact_addresspostalcode', 'contact_dob',
       'contact_idnum', 'contact_idtype', 'contact_idstate', 'contact_email',
       'application_channel', 'fraud_ind', 'fraud_tp_ind', 'fraud_fp_ind',
       'fpd_ind', 'any_wo_ind', 'activationflag', 'seq_x', 'account', 'date',
       'src', 'contact_dob_year', 'contact_dob_month', 'contact_dob_day',
       'contact_driverlicensenumber', 'credit_flag_cutoff_date',
       'credit_flag_cutoff_date_fmt', 'seq_y', 'id_seleid', 'history_datetime',
       'inq_total_count03', 'inq_total_count06', 'inq_total_

In [24]:
tmobile.fraud_ind = tmobile.fraud_ind.astype(float)
fmt = make_format(cuts = [-np.inf, 0, np.inf])
bivariate(tmobile.id_seleid, tmobile.fraud_ind, format = fmt, observed = True)

tag       fraud_ind                                               
stats             N     PctN     Sum      Mean       WoE        IV
id_seleid                                                         
<= 0         1508.0  0.02416   114.0  0.075597  0.242864  0.001587
1+          60909.0  0.97584  3648.0  0.059893 -0.006843  0.000046
Total       62417.0  1.00000  3762.0  0.060272  0.000000  0.001632

In [25]:
tmobile = tmobile[tmobile["id_seleid"] > 0]
print(tmobile.shape)

(60909, 53)


In [26]:
tmobile["credit_flag_cutoff_date"] = tmobile["appdatetime_gmt"].str.slice(0, 9)
tmobile["credit_flag_cutoff_date_fmt"] = pd.to_datetime(tmobile["credit_flag_cutoff_date"], format='%d%b%Y')

In [27]:
sbfe_for_tmobile = sbfe.merge(tmobile, left_on = "seleid_inquiry_sbfe", right_on = "id_seleid")
print(sbfe_for_tmobile.shape)
sbfe_for_tmobile = sbfe_for_tmobile[sbfe_for_tmobile["inquiry_date_fmt"] < sbfe_for_tmobile["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_tmobile.shape)

sbfe_for_tmobile['inquiry_count_06']   = np.where((sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6, 1, 0)
sbfe_for_tmobile['inquiry_count_03']   = np.where((sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3, 1, 0)
sbfe_for_tmobile['inquiry_count_01']   = np.where((sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1, 1, 0)
sbfe_for_tmobile['inquiry_count_week'] = np.where((sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1, 1, 0)

sbfe_for_tmobile_roll = sbfe_for_tmobile.groupby(by = ['crid_encr'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()
print(sbfe_for_tmobile_roll.shape)

(197067, 56)
(153107, 56)
(21683, 5)


/tmp/ipykernel_12913/1403492124.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sbfe_for_tmobile_roll = sbfe_for_tmobile.groupby(by = ['crid_encr'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()


In [28]:
tmobile = tmobile.merge(sbfe_for_tmobile_roll, on = "crid_encr", how = "left")
print(tmobile.shape)

(60909, 57)


In [29]:
tmobile['inquiry_count_06'] = np.where(tmobile["inquiry_count_06"].isnull(), 0,   tmobile["inquiry_count_06"])
tmobile['inquiry_count_03'] = np.where(tmobile["inquiry_count_03"].isnull(), 0,   tmobile["inquiry_count_03"])
tmobile['inquiry_count_01'] = np.where(tmobile["inquiry_count_01"].isnull(), 0,   tmobile["inquiry_count_01"])
tmobile['inquiry_count_week']=np.where(tmobile["inquiry_count_week"].isnull(), 0, tmobile["inquiry_count_week"])

In [30]:
tmobile["fraud_ind"] = tmobile["fraud_ind"].astype(float)

In [31]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
# keep = ['account', 'seq', 'id_seleid', 'history_datetime', 'inq_total_count03', 'inq_total_count06', 'inq_total_count12',  'inq_total_count24', 
#          'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24',]
result = dict()
keep = ['inq_total_count03', 'inq_total_count06', 'inq_total_count12',  'inq_total_count24', 'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24', 
        "inquiry_count_06", 'inquiry_count_03', 'inquiry_count_01', 'inquiry_count_week']
for i, a in enumerate(keep):
    result[i] = bivariate(tmobile[a], tmobile["fraud_ind"], format = fmt)

wb = TableWriter(filename = "./_temp/tmobile.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

In [33]:
tmobile.to_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/with_seleid/tmobile_with_seleid_and_inquiry.parquet", overwrite = True)

#### Chase

In [14]:
chase = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/Chase_11449/chase_stacked_final.parquet")
print(chase.shape)

(640958, 36)


In [15]:
chase["credit_flag_cutoff_date_fmt"] = pd.to_datetime(chase["app_date"], format='%d%b%Y')

In [16]:
keep = ['account', 'seq', 'id_seleid', 'history_datetime', 'inq_total_count', 'inq_total_count03' , 'inq_total_count06' , 'inq_total_count12', 'inq_total_count24', 
        'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24']

chase_shell = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/Chase_11449/chase_11449_bus_rep1_business_shell_v22_results_w20230201-100646_sas_layout_busshell.csv.gz", 
                           compression = "gzip", reader = pd.read_csv, encoding = 'iso-8859-1', dtype = str, usecols = keep)
print(chase_shell.shape)

(1405323, 14)


In [17]:
freq(chase["credit_flag_cutoff_date_fmt"].astype(str).str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
credit_flag_cutoff_date_fmt,,,,
2022-01,105023.0,0.163853,105023.0,0.163853
2022-02,100184.0,0.156304,205207.0,0.320157
2022-03,118369.0,0.184675,323576.0,0.504832
2022-04,107736.0,0.168086,431312.0,0.672918
2022-05,104719.0,0.163379,536031.0,0.836297
2022-06,104927.0,0.163703,640958.0,1.000000


In [18]:
chase = chase.merge(chase_shell, on = "account")
print(chase.shape)

(640958, 50)


In [19]:
chase["id_seleid"] = chase["id_seleid"].astype(float)

In [20]:
chase.fraud_ind = chase.bad.astype(float)
fmt = make_format(cuts = [-np.inf, 0, np.inf])
bivariate(chase.id_seleid, chase.bad, format = fmt, observed = True)

/tmp/ipykernel_25060/3398254818.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  chase.fraud_ind = chase.bad.astype(float)


tag             bad                                                
stats             N      PctN     Sum      Mean       WoE        IV
id_seleid                                                          
<= 0        25748.0  0.040171   274.0  0.010642  1.816945  0.372761
1+         615210.0  0.959829   844.0  0.001372 -0.240963  0.049543
Total      640958.0  1.000000  1118.0  0.001744  0.000000  0.422304

In [21]:
chase = chase[chase["id_seleid"] > 0]
print(chase.shape)

(615210, 50)


In [22]:
sbfe_for_chase = sbfe.merge(chase, left_on = "seleid_inquiry_sbfe", right_on = "id_seleid")
print(sbfe_for_chase.shape)
sbfe_for_chase = sbfe_for_chase[sbfe_for_chase["inquiry_date_fmt"] < sbfe_for_chase["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_chase.shape)

sbfe_for_chase['inquiry_count_06']   = np.where((sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6, 1, 0)
sbfe_for_chase['inquiry_count_03']   = np.where((sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3, 1, 0)
sbfe_for_chase['inquiry_count_01']   = np.where((sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1, 1, 0)
sbfe_for_chase['inquiry_count_week'] = np.where((sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1, 1, 0)

sbfe_for_chase_roll = sbfe_for_chase.groupby(by = ['account'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()
print(sbfe_for_chase_roll.shape)

(1017346, 53)
(710614, 53)


/tmp/ipykernel_25060/537911241.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sbfe_for_chase_roll = sbfe_for_chase.groupby(by = ['account'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()


(102907, 5)


In [23]:
chase = chase.merge(sbfe_for_chase_roll, on = "account", how = "left")
print(chase.shape)

(615210, 54)


In [24]:
chase['inquiry_count_06'] = np.where(chase["inquiry_count_06"].isnull(), 0,   chase["inquiry_count_06"])
chase['inquiry_count_03'] = np.where(chase["inquiry_count_03"].isnull(), 0,   chase["inquiry_count_03"])
chase['inquiry_count_01'] = np.where(chase["inquiry_count_01"].isnull(), 0,   chase["inquiry_count_01"])
chase['inquiry_count_week']=np.where(chase["inquiry_count_week"].isnull(), 0, chase["inquiry_count_week"])

In [25]:
chase["fraud_ind"] = chase["bad"].astype(float)

In [26]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
# keep = ['account', 'seq', 'id_seleid', 'history_datetime', 'inq_total_count03', 'inq_total_count06', 'inq_total_count12',  'inq_total_count24', 
#          'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24',]
result = dict()
keep = ['inq_total_count03', 'inq_total_count06', 'inq_total_count12',  'inq_total_count24', 'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24', 
        "inquiry_count_06", 'inquiry_count_03', 'inquiry_count_01', 'inquiry_count_week']
for i, a in enumerate(keep):
    result[i] = bivariate(chase[a].astype(float), chase["fraud_ind"], format = fmt)

wb = TableWriter(filename = "./_temp/chase.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

In [27]:
chase.to_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/with_seleid/chase_with_seleid_and_inquiry.parquet", overwrite = True)

#### Fleector

In [62]:
fleector = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/fleector_11124/fleector_sample_to_use.parquet")
print(fleector.shape)

(83704, 92)


In [63]:
fleector["credit_flag_cutoff_date_fmt"] = pd.to_datetime(fleector["date_created"], format='%Y-%m-%d')

In [64]:
keep = ['account', 'seq', 'id_seleid', 'history_datetime', 'inq_total_count', 'inq_total_count03' , 'inq_total_count06' , 'inq_total_count12', 'inq_total_count24', 
        'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24']

fleector_shell = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/fleector_11124/fleetcor_11124_business_shell_v22_results_w20220623-094320_sas_layout_busshell.csv", 
                           reader = pd.read_csv, encoding = 'iso-8859-1', dtype = str, usecols = keep)
print(fleector_shell.shape)

(444450, 14)


In [65]:
freq(fleector["credit_flag_cutoff_date_fmt"].astype(str).str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
credit_flag_cutoff_date_fmt,,,,
2022-01,15827.0,0.189083,15827.0,0.189083
2022-02,15512.0,0.185320,31339.0,0.374403
2022-03,19353.0,0.231208,50692.0,0.605610
2022-04,16822.0,0.200970,67514.0,0.806580
2022-05,16185.0,0.193360,83699.0,0.999940
2022-06,5.0,0.000060,83704.0,1.000000


In [66]:
fleector = fleector.merge(fleector_shell, left_on = "seq", right_on = "account")
print(fleector.shape)

(83704, 107)


In [67]:
fleector["id_seleid"] = fleector["id_seleid"].astype(float)

In [68]:
# fleector.fraud_ind = fleector.final_fraud_flag.astype(float)
fmt = make_format(cuts = [-np.inf, 0, np.inf])
bivariate(fleector.id_seleid, fleector.final_fraud_flag.astype(float), format = fmt, observed = True)

tag       final_fraud_flag                                                    
stats                    N      PctN     Sum      Mean       WoE            IV
id_seleid                                                                     
<= 0               12118.0  0.144772   975.0  0.080459  0.001019  1.502814e-07
1+                 71583.0  0.855192  5753.0  0.080368 -0.000206  3.637878e-08
Missing                3.0  0.000036     0.0  0.000000  0.000000 -0.000000e+00
Total              83704.0  1.000000  6728.0  0.080378  0.000000  1.866601e-07

In [70]:
fleector = fleector[fleector["id_seleid"] > 0]
freq('acct_flag', 'final_fraud_flag', df = fleector, cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
acct_flag final_fraud_flag                                         
0         0                 43358.0  0.605702     43358.0  0.605702
          1                  5044.0  0.070464     48402.0  0.676166
1         0                 22472.0  0.313929     70874.0  0.990095
          1                   709.0  0.009905     71583.0  1.000000

In [71]:
sbfe_for_fleector = sbfe.merge(fleector, left_on = "seleid_inquiry_sbfe", right_on = "id_seleid")
print(sbfe_for_fleector.shape)
sbfe_for_fleector = sbfe_for_fleector[sbfe_for_fleector["inquiry_date_fmt"] < sbfe_for_fleector["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_fleector.shape)

sbfe_for_fleector['inquiry_count_06']   = np.where((sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6, 1, 0)
sbfe_for_fleector['inquiry_count_03']   = np.where((sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3, 1, 0)
sbfe_for_fleector['inquiry_count_01']   = np.where((sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1, 1, 0)
sbfe_for_fleector['inquiry_count_week'] = np.where((sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1, 1, 0)

sbfe_for_fleector_roll = sbfe_for_fleector.groupby(by = ['account'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()
print(sbfe_for_fleector_roll.shape)

(251330, 110)
(136076, 110)
(28760, 5)


/tmp/ipykernel_12913/84627147.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sbfe_for_fleector_roll = sbfe_for_fleector.groupby(by = ['account'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()


In [72]:
fleector = fleector.merge(sbfe_for_fleector_roll, on = "account", how = "left")
print(fleector.shape)

(71583, 111)


In [73]:
fleector['inquiry_count_06'] = np.where(fleector["inquiry_count_06"].isnull(), 0,   fleector["inquiry_count_06"])
fleector['inquiry_count_03'] = np.where(fleector["inquiry_count_03"].isnull(), 0,   fleector["inquiry_count_03"])
fleector['inquiry_count_01'] = np.where(fleector["inquiry_count_01"].isnull(), 0,   fleector["inquiry_count_01"])
fleector['inquiry_count_week']=np.where(fleector["inquiry_count_week"].isnull(), 0, fleector["inquiry_count_week"])

In [74]:
fleector["fraud_ind"] = fleector["final_fraud_flag"].astype(float)

In [77]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
# keep = ['account', 'seq', 'id_seleid', 'history_datetime', 'inq_total_count03', 'inq_total_count06', 'inq_total_count12',  'inq_total_count24', 
#          'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24',]
result = dict()
keep = ['inq_total_count03', 'inq_total_count06', 'inq_total_count12',  'inq_total_count24', 'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24', 
        "inquiry_count_06", 'inquiry_count_03', 'inquiry_count_01', 'inquiry_count_week']
for i, a in enumerate(keep):
    result[i] = bivariate(fleector[a].astype(float), fleector["fraud_ind"], format = fmt)

wb = TableWriter(filename = "./_temp/fleector.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

In [75]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
# keep = ['account', 'seq', 'id_seleid', 'history_datetime', 'inq_total_count03', 'inq_total_count06', 'inq_total_count12',  'inq_total_count24', 
#          'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24',]
result = dict()
keep = ['inq_total_count03', 'inq_total_count06', 'inq_total_count12',  'inq_total_count24', 'inq_credit_count', 'inq_credit_count03', 'inq_credit_count06', 'inq_credit_count12', 'inq_credit_count24', 
        "inquiry_count_06", 'inquiry_count_03', 'inquiry_count_01', 'inquiry_count_week']
for i, a in enumerate(keep):
    result[i] = bivariate(fleector[a].astype(float), fleector["fraud_ind"], format = fmt, groups= fleector.acct_flag)

wb = TableWriter(filename = "./_temp/fleector.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

In [76]:
fleector.to_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/with_seleid/fleector_with_seleid_and_inquiry.parquet", overwrite = True)

In [96]:
freq("acct_flag", 'fraud_lock_flag',
 'fraud_decline_flag',
 'straight_roller_flag',
 'final_fraud_flag', df = fleector, cross = False, observed = True)

Count  \
acct_flag fraud_lock_flag fraud_decline_flag straight_roller_flag final_fraud_flag            
0         0               0                  0                    0                 53089.0   
                          1                  0                    1                  5899.0   
1         0               0                  0                    0                 23887.0   
                                             1                    1                   301.0   
                          1                  0                    1                    61.0   
          1               0                  0                    1                   451.0   
                          1                  0                    1                    16.0   

                                                                                         Pct  \
acct_flag fraud_lock_flag fraud_decline_flag straight_roller_flag final_fraud_flag             
0         0               0                  0                    0                 0.634247   
                          1                  0                    1                 0.070475   
1         0               0                  0                    0                 0.285375   
                                             1                    1                 0.003596   
                          1                  0                    1                 0.000729   
          1               0                  0                    1                 0.005388   
                          1                  0                    1                 0.000191   

                                                                                    Cuml Count  \
acct_flag fraud_lock_flag fraud_decline_flag straight_roller_flag final_fraud_flag               
0         0               0                  0                    0                    53089.0   
                          1                  0                    1                    58988.0   
1         0               0                  0                    0                    82875.0   
                                             1                    1                    83176.0   
                          1                  0                    1                    83237.0   
          1               0                  0                    1                    83688.0   
                          1                  0                    1                    83704.0   

                                                                                    Cuml Pct  
acct_flag fraud_lock_flag fraud_decline_flag straight_roller_flag final_fraud_flag            
0         0               0                  0                    0                 0.634247  
                          1                  0                    1                 0.704721  
1         0               0                  0                    0                 0.990096  
                                             1                    1                 0.993692  
                          1                  0                    1                 0.994421  
          1               0                  0                    1                 0.999809  
                          1                  0                    1                 1.000000